In [17]:
import tensorflow as tf
from numpy.random import RandomState

#4.2..2 

batch_size = 8

x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")

w1 = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)

loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y,y_),(y-y_)*loss_more,(y_-y)*loss_less))

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

rdm = RandomState(1)

dataset_size = 128

X = rdm.rand(dataset_size,2)
Y = [[x1+x2+rdm.rand()/10.0-0.05] for (x1,x2) in X]

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    Steps = 500
    for i in range(Steps):
        start = (i*batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        
        if i% 100==0:
            print("current w1 is {}".format(sess.run(w1)))


current w1 is [[-0.81031823]
 [ 1.4855988 ]]
current w1 is [[-0.7132425]
 [ 1.5827229]]
current w1 is [[-0.61900544]
 [ 1.6698811 ]]
current w1 is [[-0.5268384]
 [ 1.7513007]]
current w1 is [[-0.4349928]
 [ 1.8327824]]


In [28]:
import tensorflow as tf

#4.4.2

def get_weight(shape,lambd):
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambd)(var))
    return var

x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

batch_size = 8

layer_dimension = [2,10,10,10,2]
n_layer = len(layer_dimension)
cur_layer = x
in_dimension = layer_dimension[0]

for i in range(1,n_layer):
    out_dimension = layer_dimension[i]
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension = layer_dimension[i]
    
mes_loss = tf.reduce_mean(tf.square(y_ - cur_layer))
tf.add_to_collection('losses',mes_loss)

loss = tf.add_n(tf.get_collection('losses'))

In [38]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#5.1 5.2

#MNIST 常数
INPUT_NODE = 784 #输入节点数，即像素点个数
OUTPUT_NODE = 10 #输出点个数，0-9共10个

LAYER1_NODE = 500 #一层隐藏层，共500节点
BATCH_SIZE = 100 #一个训练Batch个数

LEARNING_RATE_BASE = 0.8 #基础学习率
LEARNING_RATE_DECAY = 0.99 #学习率的衰减率
REGULARIZATION_RATE = 0.0001 #正则化项的损失函数的系数
TRAINING_STEPS = 5000 #训练轮数
MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减率

def inference(input_tensor,avg_class,weights1,bias1,weights2,bias2):
    '''
    计算前向传播结果，定义Relu函数，实现3层全连接的神经网络
    input_tensor：输入的当前网络层的输入X或激活函数a
    avg_class：为空时，使用当前值计算；avg_class不为空时，使用传入的参数进行计算
    weights1：隐藏层的权重系数
    bias1：隐藏层的偏置函数
    weights2：输出层的权重系数
    bias2：输出层的偏置函数
    '''
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1)+bias1)
        return tf.matmul(layer1,weights2)+bias2
    else:
        #avg_class计算滑动平均值
        layer2 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(bias1))
        return tf.matmul(layer2,avg_class.average(weights2))+avg_class.average(bias2)

#模型训练
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name="x-input")
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name="y-input")
    
    #隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    bias1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    #输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    bias2 = tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算当前参数下的前向传播结果
    y = inference(x,None,weights1,bias1,weights2,bias2)
    
    
    global_step = tf.Variable(0,trainable=False)  #训练轮数（一般将训练轮数的变量指定为不可训练的参数）
    
    #给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    variable_averages_op = variable_averages.apply(tf.trainable_variables()) #在所有代表神经网络参数的变量使用滑动平均，其他辅助变量不使用
    
    #使用滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,weights1,bias1,weights2,bias2)
    
    #计算交叉熵作为预测值和真实值之间的差距作为损失函数，sparse_softmax_cross_entropy_with_logits（arg1,arg2），如果分类问题只有一个正确答案时，
    #可以用这个函数加速交叉熵的计算
    #arg1是不包括softmax层的前向传播结果，arg2是训练数据的正确答案，用tf.argmax(y_,1)得到正确答案的编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    
    #计算当前Batch中所有样例的交叉熵均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1)+regularizer(weights2)
    #总损失=交叉熵损失+正则化损失
    loss = cross_entropy_mean+regularization
    
    #设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,  #基础学习率
        global_step,         #当前迭代的轮数
        mnist.train.num_examples/BATCH_SIZE, #过完所有的训练数据需要的迭代次数
        LEARNING_RATE_DECAY) #学习率衰减速度
    
    #使用tf.train.GradientDescentOptimizer优化算法优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    #反向传播更新神经网络的参数
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    #判断使用滑动平均模型的神经网络的前向传播结果是否正确
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    
    #先将Bool转化为实数，然后计算均值
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    #初始化会话并训练
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        #准备数据
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        
        #迭代训练神经网络
        for i in range(TRAINING_STEPS):
            #每100轮输出一次结果
            if i%100 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print("After %d training steps, validation accuracy using average model is %g"%(i,validate_acc))
            
            #产生一轮的Batch训练数据，运行训练过程
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
            
        #训练结束之后，在测试数据上检测正确率   
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print("After %d training steps, Test accuracy using average model is %g"%(TRAINING_STEPS,test_acc))
        
def main(arvg=None):
    mnist = input_data.read_data_sets("/path/to/MNIST_data",one_hot = True)
    train(mnist)

#Tensorflow提供主程序入口，tf.app.run会调用main函数
if  __name__ == '__main__':
    tf.app.run()
    

Extracting /path/to/MNIST_data\train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data\train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data\t10k-labels-idx1-ubyte.gz
After 0 training steps, validation accuracy using average model is 0.0802
After 100 training steps, validation accuracy using average model is 0.9416
After 200 training steps, validation accuracy using average model is 0.957
After 300 training steps, validation accuracy using average model is 0.9646
After 400 training steps, validation accuracy using average model is 0.9694
After 500 training steps, validation accuracy using average model is 0.9712
After 600 training steps, validation accuracy using average model is 0.974
After 700 training steps, validation accuracy using average model is 0.9746
After 800 training steps, validation accuracy using average model is 0.9754
After 900 training steps, validation accuracy using average model is 0.9758
After 1000 tra

SystemExit: 

d:\code\virtualenv_jupyter_conda\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
